# Calibration of GEOtop using Nevergrad and GEOtoPy

Import stuff

In [ ]:
# Default

obs_path = '../data/Matsch B2/observations/obs.csv'
model_path = '../data/Matsch B2/inputs'
parameters_path = '../data/Calibration parameters/testbed.csv'

budget = 8192
num_workers = 256
maximum_jobs = 64

In [ ]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt

from dask.distributed import Client
from dask_jobqueue import SLURMCluster as Cluster

from mhpc_project.optimization import Parameters, Loss, NGO
from mhpc_project.utils import date_parser, comparison_plot, ProgressBar, ParametersLogger
from mhpc_project.models import SMC50
from mhpc_project.measures import KGE

Load observations and apply some filters

In [ ]:
observations = pd.read_csv(obs_path,
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)
observations.index.rename('datetime', inplace=True)

selection = observations.index.map(lambda x: not (x < date_parser('01/03/2011 23:00') or x.month in [12,1,2]))

observations = observations[selection]

Load observations, model, loss function and calibration strategy

In [ ]:
model = SMC50(model_path, timeout=180)
parameters = Parameters(parameters_path)
measure = KGE(observations['soil_moisture_50'])
loss = Loss(model, parameters, measure)
calibration = NGO(loss, budget=budget, num_workers=num_workers)

Evaluate the model and plot the results

In [ ]:
simulation = model()
print(f"Before optimization loss is {measure(simulation)}")
comparison_plot(observations['soil_moisture_50'],
                simulation,
                desc='Soil moisture content @ 5cm',
                rel=True)
plt.show()

Launch Dask cluster using `dask-jobqueue`

In [ ]:
cluster = Cluster()
cluster.adapt(maximum_jobs=maximum_jobs)

client = Client(cluster)

Run calibration

In [ ]:
calibration.optimizer.register_callback('tell', ProgressBar(total=calibration.optimizer.budget))

with warnings.catch_warnings():
    
    warnings.simplefilter("ignore")
    recommendation = calibration(executor=client)

Plot results and show table of calibrated values

In [ ]:
simulation = model(**recommendation)
print(f"After optimization loss is {measure(simulation)}")
comparison_plot(observations['soil_moisture_50'],
                simulation,
                desc='Soil moisture content @ 5cm',
                rel=True)
plt.show()

In [ ]:
calibration.to_dataframe(recommendation)